# Bring Your Own Checkpoint (BYOC)

Welcome to this tutorial on bringing your own checkpoint (BYOC) to SambaNova dedicated offerings! 

Before you get started, please follow the set up instructions given in the [README](./README.md)


## 1.  Imports

In [1]:
import sys
sys.version

'3.11.11 (main, Dec 11 2024, 10:28:39) [Clang 14.0.6 ]'

In [2]:
from IPython.display import display, HTML
display(HTML("<style>:root { --jp-notebook-max-width: 100% !important; }</style>"))
import json
import os
from dotenv import load_dotenv
from huggingface_hub import hf_hub_download, HfApi
from pathlib import Path
import pprint
load_dotenv()

True

In [3]:
ai_starter_kit_path = "../../"
snsdk_wrapper_path = ai_starter_kit_path + "utils/dedicated_env"
sys.path.append(snsdk_wrapper_path)
from snsdk_wrapper import SnsdkWrapper

## 2. Download checkpoint from HuggingFace

#### Choose your model from HuggingFace
`numind/NuExtract-1.5-tiny` is a fine-tuned version of `Qwen 2.5 0.5B` that's not currently available on Sambanova, but supported through BYOC

In [4]:
model_name = "numind/NuExtract-1.5-tiny"  

#### Specify the local directory for downloading and caching the model

In [5]:
home_dir = os.path.expanduser("~") 
cache_dir = os.path.join(home_dir + "/models")  
os.makedirs(cache_dir, exist_ok=True)

#### Download the model to the cache directory

In [6]:
repo_files = HfApi().list_repo_files(model_name)
for file_name in repo_files:    
    path = hf_hub_download(repo_id=model_name, filename=file_name, cache_dir=cache_dir)
    if file_name == "config.json":
        config_path = path
    

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.12k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/729 [00:00<?, ?B/s]

english_bench.pdf:   0%|          | 0.00/21.7k [00:00<?, ?B/s]

...

multilingual_bench.png:   0%|          | 0.00/105k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.36k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

## 3. Set your BYOC config

#### Model metadata

The metadata values can be modified manually. Note that the `param_count` is hardcoded because it does not exist in any of the HF checkpoint configs.

In [7]:
byoc = SnsdkWrapper()
byoc.config = {
    "checkpoints": [{
        "model_name": model_name.replace('/','--'),
        "checkpoint_path": str(Path(config_path).parent),
        "publisher": model_name.split('/')[0],
        "description": "BYOC - " + model_name,
        "param_count": 0.5 # number in billions of parameters
    }]
}

2025-06-05 16:20:03,007 [INFO] Using env variables to set up SNSDK and SNAPI.


#### Model parameters

Automatically extract parameters from the HuggingFace config file

In [8]:
byoc.find_config_params(verbose=False)

[{'model_arch': 'qwen2', 'seq_length': 32768, 'vocab_size': 151936}]

#### Model application

The model `app` is a grouping of models based on their architecture, vocabularly size, sequence length, and intended purpose. **If no apps are found matching the HuggingFace parameters, it is likely that the model is not yet supported on the SambaNova platform. If that's the case, please check with your SambaNova representative before proceeding with the upload.**

In [9]:
suitable_apps = byoc.get_suitable_apps(verbose=False)[0]
suitable_apps

[{'id': '12e55537-0e6a-4c3b-b591-1f6670da0d6a',
  'name': 'Samba1 Qwen2.5 Experts'}]

In [10]:
byoc.config["checkpoints"][0]["app_id"] = suitable_apps[0]["id"]

#### BYOC config summary

Review the BYOC config summary before the upload

In [11]:
pprint.pprint(byoc.config)

{'checkpoints': [{'app_id': '12e55537-0e6a-4c3b-b591-1f6670da0d6a',
                  'checkpoint_path': '/Users/varunk/models/models--numind--NuExtract-1.5-tiny/snapshots/df52efb3109d324cd52b30728f9e3fdedf19f742',
                  'description': 'BYOC - numind/NuExtract-1.5-tiny',
                  'model_arch': 'qwen2',
                  'model_name': 'numind--NuExtract-1.5-tiny',
                  'param_count': 0.5,
                  'publisher': 'numind',
                  'seq_length': 32768,
                  'vocab_size': 151936}]}


## 4. Upload checkpoint

The following command will upload the checkpoint to your dedicated environment. If you leave `verbose = True`, please note that the logs will show that the model is not found on the remote environment and upload will proceed. If the model is found, the upload will abort.

In [12]:
results = byoc.upload_checkpoints(verbose=False)

2025-06-05 16:20:39,179 [INFO] Model with name 'numind--NuExtract-1.5-tiny' not found
2025-06-05 16:20:39,319 [INFO] App with name '12e55537-0e6a-4c3b-b591-1f6670da0d6a' found with id 12e55537-0e6a-4c3b-b591-1f6670da0d6a
2025-06-05 16:20:39,320 [INFO] running snapi upload command:
 snapi import model create --model-name numind--NuExtract-1.5-tiny --app 12e55537-0e6a-4c3b-b591-1f6670da0d6a --source-type LOCAL --source-path /Users/varunk/models/models--numind--NuExtract-1.5-tiny/snapshots/df52efb3109d324cd52b30728f9e3fdedf19f742 --model-arch qwen2 --parameter-count 0.5b --sequence-length 32768 --vocab-size 151936 -ni --publisher numind --description BYOC - numind/NuExtract-1.5-tiny --ignore-transformers-version
This could take a while
2025-06-05 16:24:11,071 [INFO] Model with name 'numind--NuExtract-1.5-tiny' found with id 85f8346e-b2b2-4415-a99c-82d0ee3c55df
2025-06-05 16:24:11,075 [INFO] Model checkpoint with name 'numind--NuExtract-1.5-tiny' created it with id 85f8346e-b2b2-4415-a99c-

In [13]:
results

[{'name': 'numind--NuExtract-1.5-tiny',
  'id': '85f8346e-b2b2-4415-a99c-82d0ee3c55df'}]

You can check the status of the checkpoints after the upload process

In [14]:
status = byoc.get_checkpoints_status(results[0]["id"])[0]
status.pop("headers")
status

{'model_id': '85f8346e-b2b2-4415-a99c-82d0ee3c55df',
 'status': 'Available',
 'progress': 100,
 'stage': 'convert',
 'status_code': 200}

## Next steps

Now that you have uploaded your model checkpoint, you may want to [deploy it to an endpoint](<./Deploy a Model or Bundle to an Endpoint.ipynb>) or [create a bundle with it](<./Create a Model Bundle.ipynb>)